# Prepare the data

Divide onto train, train-valid and test

In [2]:
import shutil, os, numpy as np
from glob import glob

import torch, torchvision
from tqdm import tqdm
from zipfile import ZipFile

import cv2

In [3]:
# with ZipFile('dogs-vs-cats-redux-kernels-edition.zip') as zipf:
#     # os.mkdir('DogsVsCats')
#     zipf.extractall('DogsVsCats')

# # PERFORM ONCE
# os.mkdir(train_root+'cat')
# os.mkdir(train_root+'dog')

In [4]:
# data_root = 'DogsVsCats/'
train_root = 'Data/train/'
test_root = 'Data/test/undefined'

In [5]:
# def divide_by_classes(root, dir, src):
#     listfiles = glob(root+dir+src)
#     for f in listfiles:
#         f = f.split('\\')[-1]
#         if 'cat' in f:
#             shutil.move(root+dir+f, root+'cat/'+f)
#         else:
#             shutil.move(root+dir+f, root+'dog/'+f)

# divide_by_classes("DogsVsCats/train/", "train/", "*.jpg")

# _,_,files = next(os.walk(test_root))

# os.mkdir(test_root+'undefined')

# for f in files:
#   shutil.move(test_root+f, test_root+'undefined/'+f)

#### Define torch Datasets and transforms for data

In [7]:
IMG_SIZE = (224,224)

In [8]:
class TestDataset(torchvision.datasets.ImageFolder):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.images = glob(img_dir + '/*.jpg')
        self.transform = transform
        
    def __len__(self):
        return len(self.images)  
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.images[idx]
        img_num = img_path.split('\\')[-1].split('.')[0]
        # img_num = list(map(lambda x: x.split('\\')[-1].split('.')[0], img_path))
        # annot_path = self.annotations[idx]
        image = cv2.imread(img_path)
        # annot = transforms.ToTensor()(self.get_annot(annot_path))[0]

        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, 'annot': img_num}
        return sample

In [9]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(IMG_SIZE),
    torchvision.transforms.AutoAugment(),
    torchvision.transforms.ToTensor()
])

test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize(IMG_SIZE),
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(train_root, transform=train_transform)
# train_valid_dataset = torchvision.datasets.ImageFolder(train_valid_root, transform=train_transform)
# test_dataset = torchvision.datasets.ImageFolder(test_root, transform=test_transform)
test_dataset = TestDataset(test_root, transform=test_transform)

#### Make loaders

In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, pin_memory_device='cuda')

# train_valid_loader = torch.utils.data.DataLoader(train_valid_dataset, batch_size=128, shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, pin_memory_device='cuda')

In [12]:
len(train_loader.dataset)

25000

In [13]:
len(test_loader.dataset)

12500

In [14]:
for i in train_loader:
    print(i[0].shape, i[1].shape)
    break

torch.Size([64, 3, 224, 224]) torch.Size([64])


# Fine-tuning

In [16]:
from torchvision.models import resnet152 as model, ResNet152_Weights as model_weights

from datetime import datetime
# resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

In [17]:
class finalLayer(torch.nn.Module):
    def __init__(self, features_in):
        super(finalLayer, self).__init__()
        self.linear = torch.nn.Linear(features_in, 1)
        # self.sigm = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        # x = self.sigm(x)
        return x


def my_resnet(load=False):
  if load:
    resnet = torch.load('best_model_weights_loss.pt', weights_only=False)
  else:
    resnet = model(weights=model_weights.DEFAULT)

    resnet.fc = finalLayer(resnet.fc.in_features)
  return resnet

my_net = my_resnet(load=False)

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
my_net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
for name, param in my_net.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False
    else:
        print(name, param.requires_grad)

fc.linear.weight True
fc.linear.bias True


In [20]:
params = [p for p in my_net.parameters() if p.requires_grad]
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(
    params,
    lr=0.01
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.05
)

In [21]:
def train_one_epoch(model, optimizer, criterion, data_loader, scheduler, epoch, freq_print, prev_losses: list=[]):
    print(f"Training epoch {epoch}")
    model.train()
    data_len = len(data_loader)
    losses = prev_losses
    for X, y in tqdm(data_loader, total=data_len):
        y = y.flatten().float().cuda()
        X = X.cuda()
        yhat = model(X).flatten()
        loss = criterion(yhat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
         
        loss_improved = loss < losses[-1]
        if loss_improved:
            losses.append(loss)
            print(f'Global loss improved:{loss:.4f} Saving weights')
            torch.save(model, f'best_model_weights_loss.pt')
            
        # if (batch_idx %freq_print == 0) or (batch_idx == data_len-1):
    acc = ((yhat > .5) == y).sum() / data_loader.batch_size
            # print(f'Batch n {batch_idx}')
    print(f'Loss: {loss.item():.4f} Accuracy: {acc:.4f}')
    
    if scheduler is not None:
        scheduler.step()

    return min(losses)

In [22]:
global_losses = [100]

In [23]:
num_epochs = 3

for epoch in range(num_epochs):
    global_losses.append(train_one_epoch(my_net, optimizer, criterion, train_loader, lr_scheduler, epoch, 100, global_losses))

Training epoch 0


  0%|▏                                                                                 | 1/391 [00:00<05:30,  1.18it/s]

Global loss improved:0.7152 Saving weights


  1%|▍                                                                                 | 2/391 [00:01<04:45,  1.36it/s]

Global loss improved:0.5021 Saving weights
Global loss improved:0.3957 Saving weights


...

Global loss improved:0.0006 Saving weights


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [02:38<00:00,  2.31it/s]

Global loss improved:0.0004 Saving weights


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [02:38<00:00,  2.47it/s]

Loss: 0.0004 Accuracy: 0.6250


In [24]:
global_losses

[100,
 tensor(0.7152, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.5021, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.3957, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.2786, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.2769, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.2476, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.2227, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.1531, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.1078, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.0870, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.0672, device='cuda:0',
        grad_fn=<BinaryCrossEntropyWithLogitsBackwa

In [25]:
my_net = my_resnet(load=True)
my_net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [67]:
def make_submission(model):
    sigm = torch.nn.Sigmoid()
    result = {}
    # model.eval()
    with torch.no_grad():
        for sample in tqdm(test_loader):
            pic_names = sample['annot']
            pics = sample['image'].cuda()
            # x = x.cuda()
            output = sigm(model(pics).flatten()).tolist()
            for name, out in zip(pic_names, output):
                result[int(name)] = out
            # result += output

    return result

labels = make_submission(my_net)

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [01:11<00:00,  2.75it/s]


______

# Make submission

In [29]:
import pandas as pd

In [68]:
res_df = pd.DataFrame.from_dict(labels, 'index').sort_index().rename({0: 'label'}, axis=1)
res_df.index.name = 'id'
res_df.to_csv('submission.csv', index=True)